In [12]:
import folium
import pandas
import folium.plugins as plugins
from ipywidgets import interact

In [13]:
df1 = pandas.read_excel("Asia_2010-2019_Sep28.xlsx")

In [14]:
# extract drug-related events, year 2019
df_drug = df1.filter(["LATITUDE", 
                      "LONGITUDE", 
                      "NOTES", 
                      "FATALITIES", 
                      "YEAR"])[df1.NOTES.str.contains("\Wdrug\W", regex=True)]
df_drug = df_drug[df_drug.YEAR == 2018]

In [15]:
tiles = [name.strip() for name in """
    OpenStreetMap
    Stamen Terrain
    Stamen Toner
    Stamen Watercolor
    CartoDB positron
    CartoDB dark_matter""".strip().split('\n')]

In [16]:
@interact(years=(2016, 2019))
def create_map(years=2019):
    '''
    Creation of map embedded in interact function of the ipywidgets module.
    Efficiency is a big issue in this implementation since the datasets have 
    to be generated every time the year in the dashboard is changed.
    '''
    # data extraction
    df_drug = df1.filter(["LATITUDE", 
                      "LONGITUDE", 
                      "NOTES", 
                      "FATALITIES", 
                      "YEAR"])[df1.NOTES.str.contains("\Wdrug\W", regex=True)]
    df_drug = df_drug[df_drug.YEAR == years]
    # map initialization 
    mymap = folium.Map([21.348317, 88.602014], zoom_start=4)
    # feature group initialization
    fg_no_fatalities = folium.FeatureGroup(name='No Fatalities')
    fg_fatalities = folium.FeatureGroup(name='Fatalities')
    # marker cluster init
    marker_cluster_no_f = plugins.MarkerCluster()
    marker_cluster_f = plugins.MarkerCluster()
    # fill the marker cluster with circlemarker which are present when zooming into cluster
    for row in df_drug.iterrows():
        if row[1][3] == 0:
            marker_no_f = folium.CircleMarker(location=(row[1][0], row[1][1]),popup=row[1][2], 
                                              color="#007849", radius= 4)
            marker_cluster_no_f.add_child(marker_no_f) # no fatalities: green circle marker
        else:
            marker_f = folium.CircleMarker(location=(row[1][0], row[1][1]),popup=row[1][2], 
                                              color="#FF252B", radius= 4+row[1][3])
            marker_cluster_f.add_child(marker_f) # fatalities: red cluster marker
    # add everything to feature group and feature groups to map
    fg_no_fatalities.add_child(marker_cluster_no_f)
    fg_fatalities.add_child(marker_cluster_f)
    mymap.add_child(fg_no_fatalities)
    mymap.add_child(fg_fatalities)
    return mymap
#--------map with dashboard is not shown on nbviewer

interactive(children=(IntSlider(value=2019, description='years', max=2019, min=2016), Output()), _dom_classes=…

In [7]:
#--------map for illustration purposes on nbviewer
mymap = folium.Map([21.348317, 88.602014], zoom_start = 4)

fg_no_fatalities = folium.FeatureGroup(name='No Fatalities')
fg_fatalities = folium.FeatureGroup(name='Fatalities')

In [8]:
marker_cluster_no_f = plugins.MarkerCluster()
marker_cluster_f = plugins.MarkerCluster()
for row in df_drug.iterrows():
    if row[1][3] == 0:
        marker_no_f = folium.CircleMarker(location=(row[1][0], row[1][1]),popup=row[1][2], 
                                          color="#007849", radius= 4)
        marker_cluster_no_f.add_child(marker_no_f)
    else:
        marker_f = folium.CircleMarker(location=(row[1][0], row[1][1]),popup=row[1][2], 
                                          color="#FF252B", radius= 4+row[1][3])
        marker_cluster_f.add_child(marker_f)

In [9]:
fg_no_fatalities.add_child(marker_cluster_no_f)
fg_fatalities.add_child(marker_cluster_f)

In [11]:
mymap.add_child(fg_no_fatalities)
mymap.add_child(fg_fatalities)